# Source G-H: Vote-Based Bipartisanship Data Processing

## Overview

This notebook processes voting data from the 119th U.S. Congress to generate two foundational datasets for the Bridge Grades methodology:

- **Source G:** *Votes for Cosponsored Bills* - Counts of "yes" votes on bills with cross-party cosponsorship
- **Source H:** *Votes for Bills Sponsored by Opposing Party* - Counts of "yes" votes on bills sponsored by the opposite party

These datasets focus on voting behavior in Congress, capturing different dimensions of bipartisan alignment in roll-call votes. Specifically:

- **Source G** rewards legislators who consistently vote in favor of bills they themselves have cosponsored
- **Source H** rewards legislators who vote in favor of legislation sponsored by members of the opposite party

**Note:** As of now, these sources receive **0 weight** in the Bridge Grade due to limitations in the vote dataset — particularly, the narrow range of bills reaching a vote, often skewed by partisan agenda gatekeeping.

---

## Data Sources

### Input Files
- **`US_119_vote_people.csv`** - Individual vote records
- **`US_119_vote_counts.csv`** - Vote tallies by vote event
- **`US_119_votes.csv`** - Vote event metadata
- **`US_119_bills.csv`** - Bill information
- **`US_119_bill_sponsorships.csv`** - Bill sponsorship records
- **`119th Congress.csv`** - Legislator metadata with bioguide IDs and district assignments

### Data Source Details
- **Source:** [Plural Policy](https://open.pluralpolicy.com/data/session-csv/)
- **Congress:** 119th U.S. Congress
- **Download Date:** April 24, 2025
- **Coverage:** Individual and aggregate vote records, bill metadata, sponsorship data, and biographical information about members of Congress

---

## Outputs

### Source G: Votes for Cosponsored Bills
**File:** `bridge_grade_source_g_vote_for_cosponsored_bills.csv`

**Columns:**
- `Name`: Legislator's full name
- `bioguide_id`: Unique legislator identifier
- `Vote for Cosponsored Bill`: Raw count of "yes" votes on bills with cross-party cosponsorship
- `norm_G`: Normalized percentile score (0-100)

**Interpretation:** Higher values indicate legislators who more frequently vote in favor of bills with bipartisan cosponsorship.

### Source H: Votes for Bills Sponsored by Opposing Party
**File:** `bridge_grade_source_h_vote_for_other_party.csv`

**Columns:**
- `Name`: Legislator's full name
- `bioguide_id`: Unique legislator identifier
- `Vote for Other Party`: Raw count of "yes" votes on bills sponsored by opposite party
- `norm_H`: Normalized percentile score (0-100)

**Interpretation:** Higher values indicate legislators who more frequently vote in favor of legislation sponsored by the opposing party.

### Complete Dataset
**File:** `bridge_grade_votes_metrics_full.csv`

**Additional Columns:**
- `Party`: Legislator's political party
- `Chamber`: House or Senate
- `State`: Legislator's state
- `District`: Congressional district (House only)

---

## Technical Requirements

### Dependencies
- `pandas`: Data manipulation and analysis
- `numpy`: Numerical operations
- `scipy.stats`: Statistical functions for normalization
- `seaborn`: Statistical data visualization
- `matplotlib.pyplot`: Data visualization
- `rapidfuzz`: Fuzzy string matching
- `plotly.express`: Interactive data visualization

### Performance Notes
- Fuzzy matching is computationally intensive but necessary for data quality
- Vote aggregation operations are efficient using pandas groupby functions
- Normalization provides standardized scores for fair comparison
- All original vote records are preserved for transparency

---

## Data Quality

### Data Integrity Notes
- Vote data is filtered to "On Passage" motions only for relevance
- Senate voter IDs are recovered through name parsing when bioguide IDs are missing
- Cross-party sponsorship flags are computed at both row and bill levels
- Normalization accounts for differences in voting opportunities

### Limitations
- **Narrow Vote Range:** Only bills that reach a vote are included, which may be skewed by partisan gatekeeping
- **Missing Primary Sponsors:** 166 bills lack primary sponsor information
- **Current Weight:** These sources currently receive 0 weight in Bridge Grades due to dataset limitations

---


##  **Notebook Walkthrough: Source G-H**

### **Preprocessing for Bridge Grades: Vote-Based Bipartisanship Data**

This notebook prepares the input data used to generate two additional resources in the Bridge Grades methodology:

- **Source G:** *Votes for Cosponsored Bills*  
- **Source H:** *Votes for Bills Sponsored by Opposing Party*

These resources focus on voting behavior in Congress, capturing different dimensions of bipartisan alignment in roll-call votes. Specifically:

- **Source G** rewards legislators who consistently vote in favor of bills they themselves have cosponsored.
- **Source H** rewards legislators who vote in favor of legislation sponsored by members of the opposite party.

The datasets used in this notebook correspond to the **119th U.S. Congress** and were downloaded from the public data portal at [Plural Policy](https://open.pluralpolicy.com/data/session-csv/).

- **Date downloaded:** April 24, 2025  
- **Data includes:** Individual and aggregate vote records, bill metadata, sponsorship data, and biographical information about members of Congress.
- **Download instructions:**

1. Go to [Plural Policy](https://open.pluralpolicy.com/data/session-csv/), select "United States", then choose "United States 119th Congress".
2. Within the ZIP, the following files are used:
   - `US_119_vote_people.csv`
   - `US_119_vote_counts.csv`
   - `US_119_votes.csv`
   - `US_119_bills.csv`
   - `US_119_bill_sponsorships.csv`
   - `119th Congress.csv` (metadata with bioguide IDs and district assignments)

The notebook also includes the application of normalization and handling of missing values. As of now, these sources receive **0 weight** in the Bridge Grade due to limitations in the vote dataset — particularly, the narrow range of bills reaching a vote, often skewed by partisan gatekeeping.


In [1]:
!pip install rapidfuzz --quiet

import pandas as pd
import numpy as np
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
from rapidfuzz import process, fuzz

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))

In [2]:
#Congressional list with bioguide ids and districts
meta_data=pd.read_csv('/content/drive/MyDrive/Bridge Grades/Data/Sources G -H/input files/119th Congress.csv')

# Bill Sponsorship
sponsorships=pd.read_csv('/content/drive/MyDrive/Bridge Grades/Data/Sources G -H/input files/US_119_bill_sponsorships.csv')

# Vote Counts (by vote_id; raw counts of individual votes)
counts=pd.read_csv('/content/drive/MyDrive/Bridge Grades/Data/Sources G -H/input files/US_119_vote_counts.csv')

# Votes (vote identifier; type of vote, result of vote,d ate of vote, bill_id)
votes=pd.read_csv('/content/drive/MyDrive/Bridge Grades/Data/Sources G -H/input files/US_119_votes.csv')

# Bills (bill information)
bills=pd.read_csv('/content/drive/MyDrive/Bridge Grades/Data/Sources G -H/input files/US_119_bills.csv')

# Vote People (individual votes)
ind_votes=pd.read_csv('/content/drive/MyDrive/Bridge Grades/Data/Sources G -H/input files/US_119_vote_people.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Bridge Grades/Data/Sources G -H/input files/119th Congress.csv'

In [3]:
meta_data.head(10)

,Name,Chamber,bioguide_id,State,District,Party
0,Barry Moore,House,M001212,Alabama,1,Republican
1,Shomari Figures,House,F000481,Alabama,2,Democratic
2,Mike D. Rogers,House,R000575,Alabama,3,Republican
3,Robert B. Aderholt,House,A000055,Alabama,4,Republican
4,Dale W. Strong,House,S001220,Alabama,5,Republican
5,Gary J. Palmer,House,P000609,Alabama,6,Republican
6,Terri A. Sewell,House,S001185,Alabama,7,Democratic
7,Katie Boyd Britt,Senate,B001319,Alabama,NaN,Republican
8,Tommy Tuberville,Senate,T000278,Alabama,NaN,Republican
9,Nicholas Begich,House,B001323,Alaska,At Large,Republican


### **Match Sponsorships to Legislators**

#### **Clean and Normalize Name Fields**

In this subsection, we prepare a lowercase, punctuation-free `name_clean` column in both `meta_data` and `sponsorships` for reliable matching. We **do not** overwrite original name fields. Instead, we create new columns and:

1. Remove generational suffixes (`Jr.`, `Sr.`).  
2. Strip periods and commas in one regex operation.  
3. Trim whitespace.


In [4]:
# Remove generational suffixes and create cleaned name fields
meta_data['name_clean'] = (
    meta_data['Name']
    .str.replace(r'\b(jr|sr)\b\.?', '', case=False, regex=True)
    .str.lower()
    .str.replace(r'[.,]', '', regex=True)
    .str.strip()
)

sponsorships['name_clean'] = (
    sponsorships['name']
    .str.lower()
    .str.replace(r'[.,]', '', regex=True)
    .str.strip()
)

We now verify that the cleaning removed unwanted characters but preserved the key name components. For example, names like “John A. Smith, Jr.” become “john a smith”.


In [5]:
# Quick spot‐check
print(meta_data[['Name','name_clean']].sample(5, random_state=1))
print(sponsorships[['name','name_clean']].sample(5, random_state=1))

                Name       name_clean
363  Julie Fedorchak  julie fedorchak
426     Ralph Norman     ralph norman
255     Brad Finstad     brad finstad
306   Thomas H. Kean    thomas h kean
300  Donald Norcross  donald norcross
                     name          name_clean
12069          Adam Smith          adam smith
41171       Angus S. King        angus s king
49364      Betty McCollum      betty mccollum
574    Jefferson Van Drew  jefferson van drew
49008          Mike Carey          mike carey


#### **Split `name_clean` into Components**

Next, we decompose `name_clean` into `first_name`, `middle_name`, and `last_name` using a vectorized split:

- Use `str.split` with `n=2` to produce at most three parts.

- Fill any missing segments with empty strings.

- Truncate the first name to the first three characters for our short handle.

In [6]:
# Meta_data splitting
parts = meta_data['name_clean'].str.split(n=2, expand=True).fillna('')
parts.columns = ['first_name','middle_name','last_name']
meta_data[['first_name','middle_name','last_name']] = parts

# Sponsorships splitting
parts_s = sponsorships['name_clean'].str.split(n=2, expand=True).fillna('')
parts_s.columns = ['first_name','middle_name','last_name']
sponsorships[['first_name','middle_name','last_name']] = parts_s

# Truncate first names
meta_data['first_name'] = meta_data['first_name'].str[:3]
sponsorships['first_name'] = sponsorships['first_name'].str[:3]

#### **Build Short Name Handle**

We combine the truncated first name and the last name to create a concise `short_name` used for fuzzy matching.

In [7]:
meta_data['short_name'] = meta_data['last_name'].str.cat(meta_data['first_name'], sep=' ')
sponsorships['short_name'] = sponsorships['last_name'].str.cat(sponsorships['first_name'], sep=' ')

### **Fuzzy Match Sponsorships to Legislators**

In this section, we link each sponsorship record to a legislator by comparing `sponsorships.short_name` with `meta_data.short_name`. We:

1. Define and document a reusable fuzzy‐matching function.  
2. Apply it to generate a `matched_name`.  
3. Separate matched and unmatched records.  
4. Merge matched records to bring in legislator attributes.  
5. Ensure unmatched records retain the required schema.  
6. Concatenate into a final sponsorship table.


In [8]:
from rapidfuzz import process, fuzz

def fuzzy_merge(df_left, df_right, key_left, key_right, threshold=85):
    """
    For each value in df_left[key_left], find the best match in df_right[key_right].
    Returns a copy of df_left with a new column 'matched_name' containing the match or None.
    """
    choices = df_right[key_right].dropna().unique().tolist()
    def find_match(val):
        if pd.isna(val):
            return None
        match = process.extractOne(val, choices, scorer=fuzz.token_sort_ratio, score_cutoff=threshold)
        return match[0] if match else None

    df = df_left.copy()
    df['matched_name'] = df[key_left].apply(find_match)
    return df

# Apply fuzzy matching on the short_name handles
sponsorships_matched = fuzzy_merge(
    sponsorships, meta_data,
    key_left='short_name', key_right='short_name',
    threshold=85
)

We now have a `matched_name` for each sponsorship (or `None` if no suitable match). Next, we merge in the legislator’s details for matched records.

In [9]:
# Isolate matched and unmatched sets
matched   = sponsorships_matched[sponsorships_matched['matched_name'].notna()].copy()
unmatched = sponsorships_matched[sponsorships_matched['matched_name'].isna()].copy()

# Merge matched sponsorships with legislator attributes
matched = matched.merge(
    meta_data[['short_name','bioguide_id','Party','Chamber','State','District']],
    left_on='matched_name',
    right_on='short_name',
    how='left'
)

# For unmatched, add empty columns matching the merge schema
for col in ['bioguide_id','Party','Chamber','State','District']:
    unmatched[col] = None

Finally, we drop auxiliary columns, standardize column names, and combine matched + unmatched into the final sponsorship table.


In [14]:
# Drop temporary columns and rename for consistency
# --- Ensure sponsorship record ID and name are preserved under clear names ---
matched = matched.rename(columns={
    'id':   'sponsor_record_id',
    'name': 'sponsor_name'
})
unmatched = unmatched.rename(columns={
    'id':   'sponsor_record_id',
    'name': 'sponsor_name'
})

# --- Extract primary sponsors ---
primary_df = (
    matched
    .loc[matched['classification']=='primary',
         ['sponsor_record_id','sponsor_name','bill_id',
          'bioguide_id','Party','Chamber']]
    .rename(columns={
        'sponsor_name': 'primary_name',
        'bioguide_id':  'primary_bioguide_id',
        'Party':        'primary_party',
        'Chamber':      'primary_chamber'
    })
)

# --- Extract cosponsors ---
cosponsor_df = (
    matched
    .loc[matched['classification']=='cosponsor',
         ['sponsor_name','bill_id','bioguide_id','Party','Chamber']]
    .rename(columns={
        'sponsor_name':       'cosponsor_name',
        'bioguide_id':        'cosponsor_bioguide_id',
        'Party':              'cosponsor_party',
        'Chamber':            'cosponsor_chamber'
    })
)

matched = matched.drop(columns=[
   'name_clean','first_name','middle_name','last_name',
    'short_name_x','matched_name','short_name_y'
], errors='ignore')

unmatched = unmatched.drop(columns=[
    'name_clean','first_name','middle_name','last_name',
    'short_name','matched_name'
], errors='ignore')

# Combine into the final DataFrame
final_sponsorships = pd.concat([matched, unmatched], ignore_index=True)

# Select and reorder only the columns we need downstream
output_cols = [
    'name','bill_id','primary','classification',
    'bioguide_id','Party','Chamber','State','District'
]

# Rename sponsor_name back to name so output_cols can use 'name'
if 'sponsor_name' in final_sponsorships.columns:
    final_sponsorships = final_sponsorships.rename(columns={'sponsor_name':'name'})

final_sponsorships = final_sponsorships[output_cols]

In [11]:
final_sponsorships.head(10)

,name,bill_id,primary,classification,bioguide_id,Party,Chamber,State,District
0,David G. Valadao,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,V000129,Republican,House,California,22
1,John Garamendi,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,B001236,Republican,Senate,Arkansas,NaN
2,John Garamendi,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,G000559,Democratic,House,California,8
3,John Garamendi,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,K000393,Republican,Senate,Louisiana,NaN
4,John Garamendi,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,O000176,Democratic,House,Maryland,2
5,John Garamendi,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,J000307,Republican,House,Michigan,10
6,John Garamendi,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,M001231,Democratic,House,New York,22
7,John Garamendi,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,H001061,Republican,Senate,North Dakota,NaN
8,John Garamendi,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,J000302,Republican,House,Pennsylvania,13
9,John Garamendi,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,False,cosponsor,F000479,Democratic,Senate,Pennsylvania,NaN


### **Build Bill–Sponsor Relationships**

In this section, we construct the `bill_sponsor` DataFrame that pairs each **primary sponsor** with its **cosponsors** and flags bills that have any cross‐party support. We proceed in three subsections:

1. **Merge primary sponsors with cosponsors** (one‐to‐many).  
2. **Compute the cross‐party sponsorship flag** at the bill level.  
3. **Validate** and preview the results.  

#### **Merge Primary Sponsors with Cosponsors**

First, ensure we have exactly one primary sponsor per bill by deduplicating `primary_df` on `bill_id`. Then perform a standard left merge to attach all cosponsors.


In [15]:
# Deduplicate to one primary per bill
primary_df_unique = primary_df.drop_duplicates(subset='bill_id', keep='first')

# Merge primary sponsors (1 per bill) with all cosponsors (many per bill)
bill_sponsor = primary_df_unique.merge(
    cosponsor_df,
    on='bill_id',
    how='left'
)

#### **Compute Cross-Party Sponsorship Flag**
We create a row‐level boolean indicating whether the cosponsor’s party differs from the primary’s, then collapse this to the bill level so if any row is `True`, the flag is `True` for that bill.

In [16]:
# Row‐level flag: True if cosponsor_party ≠ primary_party
bill_sponsor['Cross Party Sponsorship'] = (
    bill_sponsor['primary_party'] != bill_sponsor['cosponsor_party']
)

# Bill‐level flag: any True within each bill_id
bill_sponsor['Cross Party Sponsorship'] = (
    bill_sponsor
    .groupby('bill_id')['Cross Party Sponsorship']
    .transform('any')
)

#### **Validate and Preview Results**

Finally, we verify that every bill has a primary sponsor, report the count of cross‐party bills, and preview the first rows.

In [17]:
# Identify bills in cosponsor_df without a matching primary
missing_primary = set(cosponsor_df['bill_id']) - set(primary_df_unique['bill_id'])
if missing_primary:
    print(f" Warning: {len(missing_primary)} bills lack a primary sponsor; they will still appear with NaNs for primary fields.")

# Summary of cross-party sponsorship
distinct_flags = bill_sponsor[['bill_id','Cross Party Sponsorship']].drop_duplicates()
num_cross = distinct_flags['Cross Party Sponsorship'].sum()
total_bills = len(distinct_flags)
print(f" {num_cross} of {total_bills} bills have cross-party cosponsors")

# Preview
bill_sponsor.head(10)


 4511 of 4929 bills have cross-party cosponsors


,sponsor_record_id,primary_name,bill_id,primary_bioguide_id,primary_party,primary_chamber,cosponsor_name,cosponsor_bioguide_id,cosponsor_party,cosponsor_chamber,Cross Party Sponsorship
0,8a396a28-95ee-4b6b-8b3a-30aeb3675121,Patrick Ryan,ocd-bill/65082e82-df48-420f-b139-443325ee91c6,R000579,Democratic,House,NaN,NaN,NaN,NaN,True
1,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,David G. Valadao,V000129,Republican,House,True
2,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,John Garamendi,B001236,Republican,Senate,True
3,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,John Garamendi,G000559,Democratic,House,True
4,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,John Garamendi,K000393,Republican,Senate,True
5,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,John Garamendi,O000176,Democratic,House,True
6,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,John Garamendi,J000307,Republican,House,True
7,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,John Garamendi,M001231,Democratic,House,True
8,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,John Garamendi,H001061,Republican,Senate,True
9,e0ee50bd-a26b-4850-9c75-5858c504c7f1,Josh Harder,ocd-bill/71f7684b-7f48-40e2-9242-0da9a644d852,H001090,Democratic,House,John Garamendi,J000302,Republican,House,True


### **Build “On Passage” Vote Table**

In this section we transform raw vote tallies (`counts`) and vote‐event metadata (`votes`, `bills`) into a clean table (`passage_votes`) containing, for each “On Passage” vote:

- **vote_id** and **bill_id**  
- Vote counts for **yes**, **no**, **abstain**, **not voting**  
- **total_votes**  
- Bill metadata (number, title, primary sponsor info)

Finally, we filter and join with our `bill_sponsor` table to attach sponsor parties and cross‐party flags.

#### **Pivot Vote Counts into Columns**

Instead of looping, we use a vectorized pivot:


In [18]:
# Drop the redundant 'id' in counts
counts = counts.drop(columns=['id'], errors='ignore')

# Pivot so each option becomes its own column
pivot_counts = (
    counts
    .pivot_table(index='vote_event_id',
                 columns='option',
                 values='value',
                 aggfunc='sum')
    .fillna(0)
    .rename_axis(None, axis=1)
    .reset_index()
)

# Standardize column names
pivot_counts.columns = [
    col.lower().replace(' ', '_') if isinstance(col, str) else col
    for col in pivot_counts.columns
]

#### **Merge with Vote Events and Bills**

We merge the pivoted tallies back into `votes`, join on `bills`, filter to “**bill”** classification, and then to **“On Passage”** motions.

In [19]:
# Merge tallies into vote events
votes = votes.merge(
    pivot_counts,
    left_on='id',
    right_on='vote_event_id',
    how='left'
)

# Attach bill details and filter to classification = bill
merged_votes_bills = (
    votes
    .merge(bills, left_on='bill_id', right_on='id', how='left', suffixes=('','_bill'))
    .query("classification == \"['bill']\"")
)

# Keep only “On Passage” motions
passage_votes = merged_votes_bills.loc[
    merged_votes_bills['motion_text'].str.startswith('On Passage', na=False),
    [
      'id', 'vote_event_id', 'bill_id',
      'yes', 'no', 'abstain', 'not_voting',
      'motion_text', 'motion_classification',
      'result', 'title'
    ]
].rename(columns={
    'id': 'vote_id',
    'vote_event_id': 'vote_event_id',
    'not_voting': 'not_voting'
})

#### **Compute Total Votes and Attach Sponsor Info**

We calculate `total_votes` as the sum of all four columns, then merge with `bill_sponsor` to bring in `primary_party`, `Cross Party Sponsorship`, etc.



In [20]:
# Total votes per event
passage_votes['total_votes'] = (
    passage_votes['yes'] +
    passage_votes['no'] +
    passage_votes['abstain'] +
    passage_votes['not_voting']
)

# Merge sponsor info
passage_votes = passage_votes.merge(
    bill_sponsor,
    on='bill_id',
    how='left'
)

# Quick check
print(f"Total passage vote records: {len(passage_votes)}")
print(f"Sum of total_votes: {passage_votes['total_votes'].sum()}")

Total passage vote records: 3274
Sum of total_votes: 1278293.0


#### **Prepare Individual Vote IDs**

Finally, we extract the list of `vote_id`s for these passage events, and filter `ind_votes` accordingly.

In [21]:
# List of valid passage vote IDs
passage_votes_ids = passage_votes['vote_id'].unique()

# Rename ind_votes field and filter
ind_votes.rename(columns={'note':'voter_bioguide_id'}, inplace=True)
ind_votes = ind_votes[ind_votes['vote_event_id'].isin(passage_votes_ids)].copy()

print(f"Total individual vote records retained: {len(ind_votes)}")

Total individual vote records retained: 7659


In [22]:
ind_votes.head(10)

,id,vote_event_id,option,voter_name,voter_id,voter_bioguide_id
1265,36656e82-50a6-428d-97da-035257cd72f9,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,yes,Rose,ocd-person/a30feb1f-c9fe-5612-808e-8fe5a55b76dc,R000612
1423,8d362275-8a0f-4bc3-828e-4eea8d0d5042,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,no,Ross,ocd-person/ea21df34-9ef4-5746-b83a-700bdc2d1918,R000305
1424,51a77ce4-1f30-403f-aac0-bd5a7fd0fd3d,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,yes,Rouzer,ocd-person/5e9726e3-2aae-5708-9227-0e95ac43c203,R000603
1425,9e56856a-4c49-4cc1-bced-2533cf35e205,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,yes,Roy,ocd-person/2398f81d-6371-544d-a7ff-24cf993a716b,R000614
1426,42ce94c0-77c7-459c-b9ad-3cc654844e4d,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,no,Ruiz,ocd-person/4db06595-7db2-51a8-8574-703e9af111b9,R000599
1427,1674538c-d27c-4db4-ae66-7003c7175f7a,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,yes,Rulli,ocd-person/91ba3b6a-a4c8-54ab-8964-8498641a6d8f,R000619
1428,5d8d3f4a-b5b2-46a6-a382-56295f4295a6,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,yes,Rutherford,ocd-person/5ab7829c-a819-5616-ac0e-a0aaa359f493,R000609
1429,d6c4d2d5-7773-48b1-9dc7-4f51aef70868,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,no,Ryan,ocd-person/10de7024-4b40-57b0-ae78-101372fd4a02,R000579
1430,c7b2e9b0-99b4-4ba1-9608-a2826f364ba1,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,yes,Salazar,ocd-person/23524e03-1206-50cb-844d-ddea47f258da,S000168
1431,0800bde7-eb50-4d91-9a3e-72c923518b05,ocd-vote/f66a86ff-3923-44ac-bb38-9ebff7500b22,no,Salinas,ocd-person/54f931b7-9a94-5c93-8585-0967370ffb86,S001226


### **Process Individual Votes & Compute G/H Metrics**

In this section we take our filtered individual‐vote records (`ind_votes`) and:

1. **Split** into House vs. Senate to recover missing Senate IDs.  
2. **Merge** with legislator metadata to attach party, chamber, etc.  
3. **Tag** each vote with its bill sponsor’s party and cross‐party flag.  
4. **Define** the two boolean metrics (Sources H & G).  
5. **Summarize** counts per legislator and **merge back** into `meta_data`.  
6. **Normalize** scores and **export** the final CSV.


#### **Split House & Senate Votes and Recover Senate IDs**

House members have full `bioguide_id` (length ≥ 5); Senate votes must be parsed from `voter_name`.



In [23]:
# Ensure no missing IDs before splitting
ind_votes = ind_votes[ind_votes['voter_bioguide_id'].notna()].copy()

# Split into House vs. Senate
house_votes  = ind_votes[ind_votes['voter_bioguide_id'].str.len() >= 5].copy()
senate_votes = ind_votes[ind_votes['voter_bioguide_id'].str.len() <  5].copy()

# Extract Last Name & State from "Name (P-XX)"
senate_votes[['Last Name','State']] = senate_votes['voter_name'] \
    .str.extract(r'^(.+?)\s*\([\w-]?([A-Z]{2})\)', expand=True)
senate_votes['Last Name'] = senate_votes['Last Name'].str.lower().str.strip()

# Build lookup of senators by last_name
meta_data['last_name'] = meta_data['Name'].str.lower().str.split().str[-1]
senate_lookup = (
    meta_data[meta_data['Chamber']=='Senate']
    .drop_duplicates(subset='last_name')
    .set_index('last_name')['bioguide_id']
)

# Map recovered IDs
senate_votes['voter_bioguide_id'] = senate_votes['Last Name'].map(senate_lookup)

# Combine back and drop auxiliaries
senate_votes = senate_votes.drop(columns=['voter_name','Last Name','State'])
all_votes   = pd.concat([house_votes, senate_votes], ignore_index=True)

#### **Merge Voter Metadata & Attach Sponsor Info**

Join each vote to the voter’s `meta_data` record, then attach the bill sponsor’s party and cross‐party flag from `passage_votes`.

In [24]:
# Merge in legislator metadata
all_votes = all_votes.merge(
    meta_data.drop(columns=['last_name']),
    left_on='voter_bioguide_id', right_on='bioguide_id',
    how='left'
)

# Attach sponsor info via merge (vectorized)
sponsor_info = passage_votes.set_index('vote_id')[['primary_party','Cross Party Sponsorship']]
all_votes = all_votes.merge(
    sponsor_info, left_on='vote_event_id', right_index=True, how='left'
).rename(columns={'primary_party':'Bill_Sponsor_Party'})

#### **Define Boolean Indicators (G & H)**

- **Source H:** Voted “yes” on a bill sponsored by a different party.

- **Source G:** Voted “yes” on a cross‐party cosponsored bill.

In [25]:
all_votes['Vote for Other Party'] = (
    (all_votes['option']=='yes') &
    (all_votes['Bill_Sponsor_Party'] != all_votes['Party'])
)

all_votes['Vote for Cosponsored Bill'] = (
    (all_votes['option']=='yes') &
    (all_votes['Cross Party Sponsorship']==True)
)

# Optional: print totals for sanity check
print("Votes for Other Party:", all_votes['Vote for Other Party'].sum())
print("Votes for Cosponsored Bill:", all_votes['Vote for Cosponsored Bill'].sum())

Votes for Other Party: 221374
Votes for Cosponsored Bill: 761337


#### **Aggregate Counts per Legislator & Merge Back**

We summarize the two boolean indicators by legislator, then merge those totals into `meta_data`. Note the correct syntax for selecting multiple columns using a list.

In [27]:
# Filter only relevant votes
cross_votes = all_votes[
    all_votes['Vote for Other Party'] |
    all_votes['Vote for Cosponsored Bill']
]

# Group and sum by Name & bioguide_id
vote_summary = (
    cross_votes
    .groupby(['Name', 'bioguide_id'])[['Vote for Other Party', 'Vote for Cosponsored Bill']]
    .sum()
    .reset_index()
)

# Merge summary back into meta_data
meta_data = meta_data.merge(
    vote_summary, on='bioguide_id', how='left'
)

# Fill NaNs with zeros for legislators with no cross-party votes
meta_data[['Vote for Other Party', 'Vote for Cosponsored Bill']] = (
    meta_data[['Vote for Other Party', 'Vote for Cosponsored Bill']]
    .fillna(0)
)

#### **Normalize Scores & Export CSVs**
Compute percentile‐style scores with the normal CDF, then save two outputs.



In [35]:
def normalize(df, col):
    m, s = df[col].mean(), df[col].std(ddof=0)
    return norm.cdf(df[col], m, s) * 100

meta_data['norm_G'] = normalize(meta_data, 'Vote for Cosponsored Bill')
meta_data['norm_H'] = normalize(meta_data, 'Vote for Other Party')

if 'Name_x' in meta_data.columns:
    meta_data.rename(columns={'Name_x':'Name'}, inplace=True)

# Export Source G
meta_data[['Name','bioguide_id','Vote for Cosponsored Bill','norm_G']] \
    .to_csv('bridge_grade_source_g_vote_for_cosponsored_bills.csv', index=False)

# Export Source H
meta_data[['Name','bioguide_id','Vote for Other Party','norm_H']] \
    .to_csv('bridge_grade_source_h_vote_for_other_party.csv', index=False)

meta_data.to_csv('bridge_grade_votes_metrics_full.csv', index=False)

print("Exported CSVs for Source G & H.")

Exported CSVs for Source G & H.


### **Visualize Metrics and Missing Data**

We’ll produce three charts using our custom `ordered_purples` palette:

1. **Histogram of raw counts** (`Vote for Cosponsored Bill`, `Vote for Other Party`)  
2. **Histogram of normalized percentiles** (`norm_G`, `norm_H`)  
3. **Bar chart of missingness** for **all** columns in `meta_data`, showing percentage on the y-axis and absolute missing count in the hover, with distinct bar outlines (gridlines between bars).



In [34]:
import plotly.express as px

# Define our custom ordered Purples palette
ordered_purples = ['#3f007d', '#54278f', '#6a51a3',
                   '#807dba', '#9e9ac8', '#bcbddc']

# 1. Raw counts distribution
raw_melt = meta_data.melt(
    id_vars=['bioguide_id','Name'],
    value_vars=['Vote for Cosponsored Bill','Vote for Other Party'],
    var_name='Metric', value_name='Count'
)

fig1 = px.histogram(
    raw_melt, x='Count', color='Metric', barmode='overlay',
    nbins=30, color_discrete_sequence=ordered_purples,
    title='Distribution of Bipartisan Vote Counts'
)
fig1.update_layout(
    plot_bgcolor='white', paper_bgcolor='white',
    legend_title_text='', xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)
fig1.show()

# 2. Percentile distribution
pct_melt = meta_data.melt(
    id_vars=['bioguide_id','Name'],
    value_vars=['norm_G','norm_H'],
    var_name='Metric', value_name='Percentile'
)

fig2 = px.histogram(
    pct_melt, x='Percentile', color='Metric', barmode='overlay',
    nbins=30, color_discrete_sequence=ordered_purples,
    title='Distribution of Bipartisan Vote Percentiles'
)
fig2.update_layout(
    plot_bgcolor='white', paper_bgcolor='white',
    legend_title_text='', xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)
fig2.show()

# 3. Missing‐value profile for all columns
missing_df = (
    meta_data
    .isna()
    .sum()
    .rename_axis('Column')
    .reset_index(name='MissingCount')
)
missing_df['MissingPct'] = missing_df['MissingCount'] / len(meta_data) * 100

fig3 = px.bar(
    missing_df, x='Column', y='MissingPct',
    text='MissingCount', color='MissingPct',
    color_continuous_scale=ordered_purples,
    title='Missing Values by Column'
)
fig3.update_traces(
    marker_line_color='black', marker_line_width=1
)
fig3.update_layout(
    plot_bgcolor='white', paper_bgcolor='white',
    yaxis_title='Missing (%)', xaxis_tickangle=-45
)
fig3.show()


### **Interpretation of Visualizations**

#### **Distribution of Bipartisan Vote Counts**
- Both metrics are highly right-skewed: most legislators cast relatively few cross-party or opposite-party “yes” votes, while a small handful have very high counts.  
- The tall bar at low counts (near zero) shows that many legislators rarely vote across party lines or on bills they didn’t sponsor.  
- The long tail—especially visible for `Vote for Other Party`—highlights a subset of highly bipartisan voters who consistently cross party lines.

#### **Distribution of Bipartisan Vote Percentiles**
- Translating raw counts into percentiles produces two distributions clustering at the extremes:  
  - A dense peak at the low end (0–20 %) for the majority who rarely cross party lines.  
  - A secondary peak at the high end (80–100 %) representing the top performers in each metric.  
- This bimodal shape underscores the polarization in voting behavior: most legislators either seldom or frequently cross the aisle, with fewer in the middle.

#### **Missing Values by Column**
- Two columns stand out with substantial missingness:  
  - **`District`** (~18 % missing): some legislators in the metadata lacked district information (e.g. Senators, delegates, or historical entries).  
  - **`Name_y`** (~19 % missing): a vestige of earlier merges where some records failed to align on the legislator’s name.  
- All other key metrics and metadata fields (party, chamber, vote counts, percentiles) are fully populated, ensuring our Bridge Grade calculations aren’t impaired by missing data in the core indicators.

Overall, these charts confirm that our metrics capture a polarized but meaningful spectrum of bipartisan activity, and that data completeness is sufficient for robust Bridge Grade scoring.